In [38]:
!pip install duckdb


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
import duckdb
import pandas as pd

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from py_duckdb.similarity_join import tokenizers
from py_duckdb.similarity_join import jaccard_join
from py_duckdb.similarity_join import jaccard_join_brute_force
from py_duckdb.similarity_join.default_names import *

In [42]:
_con = duckdb.connect(database=':memory:')

In [43]:
l_key_attr = 'id'
r_key_attr = 'id'
l_join_attr = 'purchases'
r_join_attr = 'purchases'
tokenizer = tokenizers.WhitespaceTokzr()
# tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'
tokenizer.query()

"select src, rid, len(tks) as rlen, lower(unnest(tks)) as token from ( select src, rid, list_distinct(list_filter(str_split_regex(val, '[\t\n \r]'), x -> trim(x) != '')) as tks from input ) "

In [100]:
df = pd.read_json("data/data.json", lines=True, orient='records', typ='frame')
df.head()

FileNotFoundError: File data/data.json does not exist

In [101]:
to_concat = ", ' ', ".join(["date_of_birth", "surname", "address_1", "street_number", "postcode", "soc_sec_id", "suburb", "phone_number", "state", "given_name", "age", "address_2"])
to_concat

"date_of_birth, ' ', surname, ' ', address_1, ' ', street_number, ' ', postcode, ' ', soc_sec_id, ' ', suburb, ' ', phone_number, ' ', state, ' ', given_name, ' ', age, ' ', address_2"

In [102]:
_con.execute("drop table if exists data").execute(
    "CREATE TABLE data AS "
    f"SELECT realProfileID as rid, concat ({to_concat}) as val "
    "FROM df"
).execute("select * from data").fetchall()

BinderException: Binder Error: Referenced column "realProfileID" not found in FROM clause!
Candidate bindings: "df.token"
LINE 1: CREATE TABLE data AS SELECT realProfileID as rid, concat (date_of_b...
                                    ^

In [7]:
_con.execute("drop table if exists data").execute(
    "create table data as "
    "select id as rid, interests as val "
    "from 'data/purchases.csv'"
).execute("select * from data").fetchall()

[(1, 'Bible charger pan whisk'),
 (2, 'colander comb razor tuner whisk'),
 (3, 'charger colander comb pan razor'),
 (4, 'Bible headphones tuner'),
 (5, 'headphones charger colander comb pan'),
 (6, 'razor tuner whisk')]

In [44]:
_con.execute("drop table if exists data").execute(
    "create table data as "
    "select * "
    "from 'data/purchases.csv'"
).execute("select * from data").fetchall()

[(1, 'Susan', 'Bible charger pan whisk'),
 (2, 'James', 'colander comb razor tuner whisk'),
 (3, 'Paul', 'charger colander comb pan razor'),
 (4, 'Robert', 'Bible headphones tuner'),
 (5, 'Mary', 'headphones charger colander comb pan'),
 (6, 'David', 'razor tuner whisk')]

In [46]:
jaccard_join(
    _con, 'data', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
).execute(f"select * from {out_table_name}").fetchall()

[(3, 5), (6, 2)]

In [32]:
_con.execute("describe matches").fetchall()

[('l_id', 'INTEGER', 'YES', None, None, None),
 ('r_id', 'INTEGER', 'YES', None, None, None)]

In [33]:
jaccard_join_brute_force(
    _con, 'data', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
).execute(f"select * from {out_table_name}").fetchall()

[(2, 6), (3, 5)]

In [34]:
l_table = "data"

In [35]:
_con.execute(f"drop table if exists {INPUT_TABLE}").execute(
    f"create table {INPUT_TABLE} as ("
    f"select '{l_table}' as src, {l_key_attr} as rid, {l_join_attr} as val "
    f"from '{l_table}' )"
).execute(f"select * from {INPUT_TABLE}").fetchall()

[('data', 1, 'Bible charger pan whisk'),
 ('data', 2, 'colander comb razor tuner whisk'),
 ('data', 3, 'charger colander comb pan razor'),
 ('data', 4, 'Bible headphones tuner'),
 ('data', 5, 'headphones charger colander comb pan'),
 ('data', 6, 'razor tuner whisk')]

In [36]:
_con.execute(f"drop table if exists {TOKENS_VIEW}").execute(
    f"create table {TOKENS_VIEW} as "
    + tokenizer.query()
).execute(f"select * from {TOKENS_VIEW}").fetchall()

[('data', 1, 4, 'bible'),
 ('data', 1, 4, 'charger'),
 ('data', 1, 4, 'pan'),
 ('data', 1, 4, 'whisk'),
 ('data', 2, 5, 'colander'),
 ('data', 2, 5, 'comb'),
 ('data', 2, 5, 'whisk'),
 ('data', 2, 5, 'razor'),
 ('data', 2, 5, 'tuner'),
 ('data', 3, 5, 'charger'),
 ('data', 3, 5, 'colander'),
 ('data', 3, 5, 'pan'),
 ('data', 3, 5, 'comb'),
 ('data', 3, 5, 'razor'),
 ('data', 4, 3, 'bible'),
 ('data', 4, 3, 'headphones'),
 ('data', 4, 3, 'tuner'),
 ('data', 5, 5, 'headphones'),
 ('data', 5, 5, 'charger'),
 ('data', 5, 5, 'colander'),
 ('data', 5, 5, 'pan'),
 ('data', 5, 5, 'comb'),
 ('data', 6, 3, 'whisk'),
 ('data', 6, 3, 'razor'),
 ('data', 6, 3, 'tuner')]

In [37]:
_con.execute(f"drop table if exists {DOC_FREQ_VIEW}").execute(
    f"CREATE table {DOC_FREQ_VIEW} AS "
    "SELECT token, count(*) AS df "
    f"FROM {TOKENS_VIEW} "
    "GROUP BY token "
).execute(f"drop table if exists {TOKENS_DOC_FREQ_VIEW}").execute(
    f"CREATE table {TOKENS_DOC_FREQ_VIEW} AS "
    f"select rid, rlen, {TOKENS_VIEW}.token, df "
    f", row_number() OVER (PARTITION BY rid ORDER BY df, {TOKENS_VIEW}.token) as pos "
    f", concat(rlen, '_', rid) as lrid "
    f"from {TOKENS_VIEW}, {DOC_FREQ_VIEW} "
    f"where {TOKENS_VIEW}.token = {DOC_FREQ_VIEW}.token"
).execute(f"select * from {TOKENS_DOC_FREQ_VIEW}").fetchall()

[(1, 4, 'bible', 2, 1, '4_1'),
 (1, 4, 'charger', 3, 2, '4_1'),
 (1, 4, 'pan', 3, 3, '4_1'),
 (1, 4, 'whisk', 3, 4, '4_1'),
 (2, 5, 'colander', 3, 1, '5_2'),
 (2, 5, 'comb', 3, 2, '5_2'),
 (2, 5, 'razor', 3, 3, '5_2'),
 (2, 5, 'tuner', 3, 4, '5_2'),
 (2, 5, 'whisk', 3, 5, '5_2'),
 (3, 5, 'charger', 3, 1, '5_3'),
 (3, 5, 'colander', 3, 2, '5_3'),
 (3, 5, 'comb', 3, 3, '5_3'),
 (3, 5, 'pan', 3, 4, '5_3'),
 (3, 5, 'razor', 3, 5, '5_3'),
 (4, 3, 'bible', 2, 1, '3_4'),
 (4, 3, 'headphones', 2, 2, '3_4'),
 (4, 3, 'tuner', 3, 3, '3_4'),
 (5, 5, 'headphones', 2, 1, '5_5'),
 (5, 5, 'charger', 3, 2, '5_5'),
 (5, 5, 'colander', 3, 3, '5_5'),
 (5, 5, 'comb', 3, 4, '5_5'),
 (5, 5, 'pan', 3, 5, '5_5'),
 (6, 3, 'razor', 3, 1, '3_6'),
 (6, 3, 'tuner', 3, 2, '3_6'),
 (6, 3, 'whisk', 3, 3, '3_6')]

In [34]:
_con.execute(f"select * from {TOKENS_DOC_FREQ_VIEW}").fetchall()

[(1, 4, 'bible', 2, 1, '4_1'),
 (1, 4, 'charger', 3, 2, '4_1'),
 (1, 4, 'pan', 3, 3, '4_1'),
 (1, 4, 'whisk', 3, 4, '4_1'),
 (2, 5, 'headphones', 2, 1, '5_2'),
 (2, 5, 'charger', 3, 2, '5_2'),
 (2, 5, 'colander', 3, 3, '5_2'),
 (2, 5, 'comb', 3, 4, '5_2'),
 (2, 5, 'pan', 3, 5, '5_2'),
 (3, 5, 'colander', 3, 1, '5_3'),
 (3, 5, 'comb', 3, 2, '5_3'),
 (3, 5, 'razor', 3, 3, '5_3'),
 (3, 5, 'tuner', 3, 4, '5_3'),
 (3, 5, 'whisk', 3, 5, '5_3'),
 (4, 5, 'charger', 3, 1, '5_4'),
 (4, 5, 'colander', 3, 2, '5_4'),
 (4, 5, 'comb', 3, 3, '5_4'),
 (4, 5, 'pan', 3, 4, '5_4'),
 (4, 5, 'razor', 3, 5, '5_4'),
 (5, 3, 'bible', 2, 1, '3_5'),
 (5, 3, 'headphones', 2, 2, '3_5'),
 (5, 3, 'tuner', 3, 3, '3_5'),
 (6, 3, 'razor', 3, 1, '3_6'),
 (6, 3, 'tuner', 3, 2, '3_6'),
 (6, 3, 'whisk', 3, 3, '3_6')]

In [35]:
t = threshold

In [36]:
_con.execute(f"drop table if exists {PREFIXES_VIEW}").execute(
    f"create table {PREFIXES_VIEW} as "
    "select rid, rlen, token, pos, df "
    f"from {TOKENS_DOC_FREQ_VIEW} "
    f"where rlen - pos + 1 >= ceil(rlen * {t}) "
).execute(f"select * from {PREFIXES_VIEW}").fetchall()

[(1, 4, 'bible', 1, 2),
 (1, 4, 'charger', 2, 3),
 (1, 4, 'pan', 3, 3),
 (2, 5, 'headphones', 1, 2),
 (2, 5, 'charger', 2, 3),
 (2, 5, 'colander', 3, 3),
 (3, 5, 'colander', 1, 3),
 (3, 5, 'comb', 2, 3),
 (3, 5, 'razor', 3, 3),
 (4, 5, 'charger', 1, 3),
 (4, 5, 'colander', 2, 3),
 (4, 5, 'comb', 3, 3),
 (5, 3, 'bible', 1, 2),
 (5, 3, 'headphones', 2, 2),
 (6, 3, 'razor', 1, 3),
 (6, 3, 'tuner', 2, 3)]

In [37]:
_con.execute(f"drop table if exists {CANDIDATE_SET_VIEW}").execute(
    f"CREATE table {CANDIDATE_SET_VIEW} AS "
    "SELECT pr1.rid AS rid1, pr2.rid AS rid2 "
    ", MAX(pr1.pos) as maxPos1, MAX(pr2.pos) as maxPos2, count(*) as prOverlap "
    f"FROM {TOKENS_DOC_FREQ_VIEW} pr1, {TOKENS_DOC_FREQ_VIEW} pr2 "
    "WHERE pr1.lrid < pr2.lrid "
    "AND pr1.token = pr2.token "
    # length filter
    f"AND pr1.rlen >= (pr2.rlen * {t})"
    # prefix filter
    f"AND pr1.rlen - pr1.pos + 1 >= CEIL(pr1.rlen * 2 * {t} / (1+{t})) "
    f"AND pr2.rlen - pr2.pos + 1 >= (pr2.rlen * {t}) "
    # positional filter
    "AND LEAST((pr1.rlen - pr1.pos + 1), (pr2.rlen - pr2.pos + 1)) >= "
    f"CEIL((pr1.rlen + pr2.rlen) * {t} / (1 + {t})) "
    "GROUP BY pr1.rid, pr2.rid "
).execute(f"select * from {CANDIDATE_SET_VIEW}").fetchall()

[(1, 2, 2, 2, 1),
 (1, 4, 2, 1, 1),
 (2, 4, 2, 1, 1),
 (3, 4, 1, 2, 1),
 (5, 1, 1, 1, 1),
 (6, 3, 1, 3, 1)]

In [38]:
_con.execute(
    # Start from the last match included to include the pairs in which the prefixes match entirely but the
    # suffixes do not match at all
    "select r1.rid as rid1, r2.rid as rid2 "
    # f", count(*) + prOverlap - 1, (r1.rlen + r2.rlen) * {t} / (1+{t}) "
    f"from {TOKENS_DOC_FREQ_VIEW} r1, {TOKENS_DOC_FREQ_VIEW} r2, {CANDIDATE_SET_VIEW} c "
    "where c.rid1 = r1.rid "
    "and c.rid2 = r2.rid "
    "and r1.token = r2.token "
    "and r1.pos >= maxPos1 "# "and r1.pos > maxPos1 "
    "and r2.pos >= maxPos2 "# "and r2.pos > maxPos2 "
    "group by r1.rid, r2.rid, r1.rlen, r2.rlen, prOverlap "
    f"having count(*) + prOverlap - 1 >= (r1.rlen + r2.rlen) * {t} / (1+{t})"
).fetchall()

[(2, 4), (6, 3)]

In [39]:
_con.execute(
    "select r1.rid as rid1, r2.rid as rid2 "
    # f", count(*) as overlap "
    # f", ceil({t} / (1+{t}) * (r1.rlen + r2.rlen)) "
    f"from {TOKENS_VIEW} as r1, {TOKENS_VIEW} as r2 "
    "where r1.token = r2.token "
    "and r1.rid < r2.rid "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    # f"having count(*) >= ({t} / (1+{t}) * (r1.rlen + r2.rlen))" # -> 5 >= 5.000000001 KO
    f"having count(*) >= ((r1.rlen + r2.rlen) * {t} / (1+{t}))" # -> 5 >= 5.0 OK
).fetchall()

[(2, 4), (3, 6)]

### Calcolo "a mano" la similarità delle coppie presenti nel Ground Truth

In [7]:
df10id = pd.read_json("data/10KIdDuplicates.json", lines=True, orient='records', typ='frame')
_con.execute("drop table if exists db10id").execute(
    "CREATE TABLE db10id AS "
    "SELECT d1Id as rid1, d2Id as rid2 "
    "FROM df10id "
).execute("select count(*) from db10id").fetchall()

[(8705,)]

In [8]:
df10 = pd.read_json("data/10Kprofiles.json", lines=True, orient='records', typ='frame')
_con.execute("drop table if exists db10").execute(
    "CREATE TABLE db10 AS "
    "SELECT realProfileID as rid, concat (date_of_birth, '|', surname, '|', address_1, '|', street_number, '|', postcode, '|',soc_sec_id, '|',"
    "suburb, '|',phone_number, '|', state, '|', given_name, '|',age, '|',address_2 ) as val "
    "FROM df10"
).execute("select * from db10").fetchall()

[(0,
  '19390609.0|bishop|daley crescent|41.0|6050|4676841|batlow|08 29028996|qld|molly|31.0|'),
 (1,
  '19041109.0|aidon|nambucca street|7.0|2002|3414163|devonort|08 75629459|vkf|whkt||'),
 (2,
  '19910711.0|anns||12.0|2287|7844876|ivanhoe|02 11684110|vic|andrew|31.0|'),
 (3,
  '19390709.0|whitrlsy|robson street|34.0|4065|2418360|christie downs|04 00323207|qld|shsne||'),
 (4,
  '19340328.0|roche|rankin street|9.0|3644|7577436|frenchs forest|08 53227250|qld|sophie|30.0|'),
 (5,
  '19320811.0|fullgrabe|beeston street|29.0|3131|6494586|broken hill|04 80080021|nsw|emma|29.0|'),
 (6,
  '19601013.0|lodge|mason street|48.0|5254|6098877|orchard hills|08 48143359|vic|rourke|32.0|'),
 (7,
  '19340921.0|coleman|edman close|116.0|2397|8858237|castlecrag|03 58778382||harry|26.0|'),
 (8,
  '19671108.0|bishop|cromwell circuit|12.0|2226|1718686|harris park|04 01707833|vic|jamie|28.0|homestead caravan park'),
 (9,
  '19331228.0|grcn|lutana street|11.0|2074|8364236|newstead|02 57700508|ws|lauren||'),
 

In [10]:
l_table = "db10"
tokenizer = tokenizers.WordsTokzr("'\|'")

_con.execute(f"drop table if exists {INPUT_TABLE}").execute(
    f"create table {INPUT_TABLE} as ("
    f"select '{l_table}' as src, {l_key_attr} as rid, {l_join_attr} as val "
    f"from '{l_table}' )"
).execute(f"select * from {INPUT_TABLE}").fetchall()
_con.execute(f"drop table if exists {TOKENS_VIEW}").execute(
    f"create table {TOKENS_VIEW} as "
    + tokenizer.query()
).execute(f"select * from {TOKENS_VIEW}").fetchall()

[('db10', 0, 11, '19390609.0'),
 ('db10', 0, 11, '41.0'),
 ('db10', 0, 11, 'bishop'),
 ('db10', 0, 11, 'daley crescent'),
 ('db10', 0, 11, '6050'),
 ('db10', 0, 11, '4676841'),
 ('db10', 0, 11, 'batlow'),
 ('db10', 0, 11, '08 29028996'),
 ('db10', 0, 11, 'qld'),
 ('db10', 0, 11, 'molly'),
 ('db10', 0, 11, '31.0'),
 ('db10', 1, 10, 'aidon'),
 ('db10', 1, 10, '19041109.0'),
 ('db10', 1, 10, 'nambucca street'),
 ('db10', 1, 10, '08 75629459'),
 ('db10', 1, 10, 'devonort'),
 ('db10', 1, 10, '7.0'),
 ('db10', 1, 10, '3414163'),
 ('db10', 1, 10, '2002'),
 ('db10', 1, 10, 'vkf'),
 ('db10', 1, 10, 'whkt'),
 ('db10', 2, 10, '12.0'),
 ('db10', 2, 10, '19910711.0'),
 ('db10', 2, 10, 'vic'),
 ('db10', 2, 10, 'anns'),
 ('db10', 2, 10, '2287'),
 ('db10', 2, 10, '7844876'),
 ('db10', 2, 10, '02 11684110'),
 ('db10', 2, 10, 'ivanhoe'),
 ('db10', 2, 10, 'andrew'),
 ('db10', 2, 10, '31.0'),
 ('db10', 3, 10, 'christie downs'),
 ('db10', 3, 10, 'robson street'),
 ('db10', 3, 10, '19390709.0'),
 ('db10', 3

In [15]:
_con.execute(
    "select * , ("
    "select count(*) as overlap "
    f"from {TOKENS_VIEW} t1, {TOKENS_VIEW} t2 "
    "where t1.rid = id.rid1 "
    "and t2.rid = id.rid2 "
    "and t1.token = t2.token "
    ")"
    f"from db10id id "
).fetchall()

[(101, 8933, 8),
 (4101, 4614, 5),
 (7213, 8466, 4),
 (4856, 7485, 2),
 (4829, 9725, 6),
 (1289, 8634, 5),
 (3059, 9179, 3),
 (6802, 9806, 3),
 (6218, 9126, 4),
 (5849, 6986, 9),
 (3119, 4205, 4),
 (1884, 8409, 6),
 (5663, 6060, 4),
 (9371, 9596, 6),
 (8195, 8902, 9),
 (1295, 8153, 5),
 (4301, 4419, 4),
 (5059, 7040, 5),
 (2498, 6623, 2),
 (2734, 3420, 2),
 (3670, 7648, 7),
 (358, 4020, 7),
 (3064, 8794, 4),
 (772, 2425, 7),
 (389, 1449, 6),
 (7025, 7724, 4),
 (1689, 8239, 4),
 (557, 3895, 5),
 (1554, 3063, 3),
 (3699, 5250, 3),
 (2924, 4049, 7),
 (5656, 6673, 5),
 (8975, 9725, 3),
 (3875, 7035, 9),
 (2704, 5934, 5),
 (4040, 9733, 6),
 (895, 8626, 3),
 (3317, 4217, 6),
 (1328, 5463, 5),
 (184, 2109, 4),
 (96, 9418, 7),
 (781, 1715, 4),
 (4486, 5499, 2),
 (3252, 9615, 3),
 (4858, 7396, 4),
 (2129, 4535, 4),
 (1083, 9433, 4),
 (3267, 8384, 5),
 (8392, 8991, 1),
 (3105, 5454, 3),
 (1921, 5428, 4),
 (6033, 8204, 3),
 (1483, 9012, 7),
 (1282, 9311, 6),
 (6822, 8240, 2),
 (303, 8653, 7),
 (4

In [24]:
_con.execute(
    "select id.rid1, id.rid2, count(*) as overlap, overlap * 100 / (any_value(t1.rlen) + (any_value(t2.rlen) - overlap)) as jaccard "
    f"from db10id id, {TOKENS_VIEW} t1, {TOKENS_VIEW} t2 "
    "where t1.rid = id.rid1 "
    "and t2.rid = id.rid2 "
    "and t1.token = t2.token "
    "group by id.rid1, id.rid2 "
    "order by jaccard"
).fetchall()

[(14, 7100, 1, 4),
 (1116, 2239, 1, 4),
 (1598, 6100, 1, 4),
 (2024, 2905, 1, 4),
 (2149, 4857, 1, 4),
 (2521, 5886, 1, 4),
 (2521, 5597, 1, 4),
 (2717, 6146, 1, 4),
 (4387, 7835, 1, 4),
 (4809, 8982, 1, 4),
 (6113, 8747, 1, 4),
 (6123, 7110, 1, 4),
 (7184, 7789, 1, 4),
 (5807, 9610, 1, 4),
 (5546, 6330, 1, 4),
 (4882, 8659, 1, 4),
 (45, 7205, 1, 5),
 (54, 9209, 1, 5),
 (127, 7809, 1, 5),
 (1, 8684, 1, 5),
 (12, 4572, 1, 5),
 (59, 5230, 1, 5),
 (120, 5616, 1, 5),
 (152, 4082, 1, 5),
 (291, 5443, 1, 5),
 (502, 9203, 1, 5),
 (471, 8605, 1, 5),
 (567, 1758, 1, 5),
 (654, 7490, 1, 5),
 (663, 4705, 1, 5),
 (753, 7522, 1, 5),
 (651, 705, 1, 5),
 (705, 2452, 1, 5),
 (594, 6272, 1, 5),
 (814, 4377, 1, 5),
 (877, 8991, 1, 5),
 (882, 8338, 1, 5),
 (890, 8279, 1, 5),
 (907, 8108, 1, 5),
 (965, 9331, 1, 5),
 (913, 5333, 1, 5),
 (927, 5850, 1, 5),
 (943, 4093, 1, 5),
 (796, 1090, 1, 5),
 (1040, 9642, 1, 5),
 (1124, 9795, 1, 5),
 (1052, 7152, 1, 5),
 (1083, 6969, 1, 5),
 (538, 3676, 1, 5),
 (705, 46

## Profiles, non-self join

In [34]:
df10 = pd.read_json("data/10Kprofiles.json", lines=True, orient='records', typ='frame')
split_at = int(df10.shape[0] / 2)
df10_1 = df10.iloc[:split_at, :]
df10_2 = df10.iloc[split_at:, :]
to_concat = ", ' ', ".join(
    ["date_of_birth", "surname", "address_1", "street_number", "postcode", "soc_sec_id", "suburb", "phone_number",
     "state", "given_name", "age", "address_2"])
_con.execute("drop table if exists db10_1").execute(
    "CREATE TABLE db10_1 AS "
    f"SELECT realProfileID as rid, concat ({to_concat}) as val "
    "FROM df10_1"
).execute("select * from db10_1").fetchall()
_con.execute("drop table if exists db10_2").execute(
    "CREATE TABLE db10_2 AS "
    f"SELECT realProfileID as rid, concat ({to_concat}) as val "
    "FROM df10_2"
).execute("select * from db10_2").fetchall()

[(5000,
  '19700823.0 wang  8.0 2875 2216347 blue haven 07 17987225  ameli 32.0 '),
 (5001,
  '19280314.0 scaror warramoo crescent 7.0 5752 4933901 moama 08 76672148 vl kiara 23.0 '),
 (5002,
  '19290324.0 colis alarmon crescent 79.0 3204 5111085 warren 03 66459383 act georgia 28.0 arthella'),
 (5003,
  ' wilkins learmonth drive 24.0 2050 5694132 miami 07 04918885 qld benjamin 21.0 hopeview'),
 (5004,
  '19240216.0 fraenkel launceston street 44.0 2088 3703441 dapto 08 57499031  timothy 30.0 '),
 (5005,
  '19120121.0 rees achernar close 61.0 7011 6346462 dapto 02 97724353 nsw kane 36.0 retirement village'),
 (5006,
  '19051490.0 binns wenholz treet  6006 2604157 dapto 03 68821645 sa lucinda 29.0 '),
 (5007,
  ' mccarthy millhouse crescent 35.0 6027 8045670 northmead 08 76825732 qld sophie 30.0 highgate'),
 (5008,
  '19250109.0 hearn stapylton street 1.0 6027 2267622 malvern east 03 64393123  tynan 29.0 '),
 (5009,
  '19791026.0 stuber saville close 75.0 4350 3716864 hay 02 96356289 nsw 

In [35]:
df10gt = pd.read_json("data/10KIdDuplicates.json", lines=True, orient='records', typ='frame')
_con.execute("drop table if exists db10gt").execute(
    "CREATE TABLE db10gt AS "
    "SELECT d1Id as rid1, d2Id as rid2 "
    "FROM df10gt "
    f"where d1Id < {split_at} and d2Id >= {split_at} "
).execute("select * from db10gt").fetchall()

[(101, 8933),
 (4856, 7485),
 (4829, 9725),
 (1289, 8634),
 (3059, 9179),
 (1884, 8409),
 (1295, 8153),
 (2498, 6623),
 (3670, 7648),
 (3064, 8794),
 (1689, 8239),
 (3699, 5250),
 (3875, 7035),
 (2704, 5934),
 (4040, 9733),
 (895, 8626),
 (1328, 5463),
 (96, 9418),
 (4486, 5499),
 (3252, 9615),
 (4858, 7396),
 (1083, 9433),
 (3267, 8384),
 (3105, 5454),
 (1921, 5428),
 (1483, 9012),
 (1282, 9311),
 (303, 8653),
 (4884, 5268),
 (687, 9557),
 (90, 9958),
 (4631, 9892),
 (1488, 8627),
 (1488, 8625),
 (3302, 5519),
 (4058, 8307),
 (2297, 7018),
 (701, 8420),
 (1299, 7941),
 (3065, 8829),
 (3454, 9318),
 (1502, 7488),
 (2860, 9462),
 (3279, 7464),
 (516, 7416),
 (3488, 6510),
 (3690, 6123),
 (3273, 7966),
 (2092, 7690),
 (2502, 6433),
 (2909, 5433),
 (1477, 9595),
 (684, 9881),
 (880, 9997),
 (4492, 5118),
 (3065, 8883),
 (2107, 6474),
 (4066, 7719),
 (511, 7876),
 (2118, 5569),
 (3850, 9273),
 (2878, 8024),
 (3087, 7062),
 (1510, 6894),
 (1890, 8133),
 (2117, 5676),
 (3282, 7290),
 (3676, 

In [36]:
l_table = 'db10_1'
r_table = 'db10_2'

In [37]:
l_count = _con.execute(
    "select count(*) "
    f"from {l_table}"
).fetchall()[0][0]
r_count = _con.execute(
    "select count(*) "
    f"from {r_table}"
).fetchall()[0][0]

In [38]:
_con.execute(
    f"drop table if exists {INPUT_TABLE}"
).execute(
    f"create table {INPUT_TABLE} as ("
    f"select '{l_table}' as src, {l_key_attr} as rid, {l_join_attr} as val "
    f"from '{l_table}' "
    "union "
    f"select '{r_table}' as src, {r_key_attr} as rid, {r_join_attr} as val "
    f"from '{r_table}' )"
).execute(f'select * from {INPUT_TABLE}').fetchall()

[('db10_1',
  0,
  '19390609.0 bishop daley crescent 41.0 6050 4676841 batlow 08 29028996 qld molly 31.0 '),
 ('db10_1',
  1,
  '19041109.0 aidon nambucca street 7.0 2002 3414163 devonort 08 75629459 vkf whkt  '),
 ('db10_1',
  2,
  '19910711.0 anns  12.0 2287 7844876 ivanhoe 02 11684110 vic andrew 31.0 '),
 ('db10_1',
  3,
  '19390709.0 whitrlsy robson street 34.0 4065 2418360 christie downs 04 00323207 qld shsne  '),
 ('db10_1',
  4,
  '19340328.0 roche rankin street 9.0 3644 7577436 frenchs forest 08 53227250 qld sophie 30.0 '),
 ('db10_1',
  5,
  '19320811.0 fullgrabe beeston street 29.0 3131 6494586 broken hill 04 80080021 nsw emma 29.0 '),
 ('db10_1',
  6,
  '19601013.0 lodge mason street 48.0 5254 6098877 orchard hills 08 48143359 vic rourke 32.0 '),
 ('db10_1',
  7,
  '19340921.0 coleman edman close 116.0 2397 8858237 castlecrag 03 58778382  harry 26.0 '),
 ('db10_1',
  8,
  '19671108.0 bishop cromwell circuit 12.0 2226 1718686 harris park 04 01707833 vic jamie 28.0 homestead c

In [39]:
_con.execute(f"drop table if exists {TOKENS_VIEW}").execute(
    f"create table {TOKENS_VIEW} as "
    + tokenizer.query()
).execute(f"select * from {TOKENS_VIEW}").fetchall()

[('db10_1', 0, 13, '19390609.0'),
 ('db10_1', 0, 13, '41.0'),
 ('db10_1', 0, 13, 'bishop'),
 ('db10_1', 0, 13, 'daley'),
 ('db10_1', 0, 13, 'batlow'),
 ('db10_1', 0, 13, 'crescent'),
 ('db10_1', 0, 13, '29028996'),
 ('db10_1', 0, 13, '6050'),
 ('db10_1', 0, 13, '4676841'),
 ('db10_1', 0, 13, '08'),
 ('db10_1', 0, 13, 'qld'),
 ('db10_1', 0, 13, 'molly'),
 ('db10_1', 0, 13, '31.0'),
 ('db10_1', 1, 12, 'aidon'),
 ('db10_1', 1, 12, '19041109.0'),
 ('db10_1', 1, 12, 'nambucca'),
 ('db10_1', 1, 12, 'street'),
 ('db10_1', 1, 12, 'devonort'),
 ('db10_1', 1, 12, '7.0'),
 ('db10_1', 1, 12, '3414163'),
 ('db10_1', 1, 12, '2002'),
 ('db10_1', 1, 12, '08'),
 ('db10_1', 1, 12, '75629459'),
 ('db10_1', 1, 12, 'vkf'),
 ('db10_1', 1, 12, 'whkt'),
 ('db10_1', 2, 11, '12.0'),
 ('db10_1', 2, 11, '19910711.0'),
 ('db10_1', 2, 11, 'anns'),
 ('db10_1', 2, 11, '2287'),
 ('db10_1', 2, 11, '02'),
 ('db10_1', 2, 11, '7844876'),
 ('db10_1', 2, 11, '11684110'),
 ('db10_1', 2, 11, 'ivanhoe'),
 ('db10_1', 2, 11, 'vi

In [106]:
t = threshold

In [107]:
widow_placeholder = l_count * r_count + 1

In [108]:
_con.execute(
    f"drop table if exists full_outer_{DOC_FREQ_VIEW}"
).execute(
    f"create table full_outer_{DOC_FREQ_VIEW} as "
    "select l_tks.token as l_tk, l_tks.df as l_df, r_tks.token as r_tk, r_tks.df as r_df "
    "from ("
    "SELECT token, count(*) AS df "
    f"FROM {TOKENS_VIEW} "
    f"where src = '{l_table}' "
    "GROUP BY token "
    ") as l_tks "
    "full outer join ("
    "SELECT token, count(*) AS df "
    f"FROM {TOKENS_VIEW} "
    f"where src = '{r_table}' "
    "GROUP BY token "
    ") as r_tks "
    "on l_tks.token = r_tks.token"
).execute(
    f"drop table if exists {DOC_FREQ_VIEW}"
).execute(
    # Include widows, with df=null
    f"create table {DOC_FREQ_VIEW} as ("
    f"select coalesce(l_tk, r_tk) as token, coalesce(l_df * r_df, {widow_placeholder}) as df "
    f"from full_outer_{DOC_FREQ_VIEW} "
    ")"
).execute(
    f"drop table if exists {TOKENS_DOC_FREQ_VIEW}"
).execute(
    # Arbitrary high df to widows, to preserve them in the table but not in the prefixes
    f"create table {TOKENS_DOC_FREQ_VIEW} as "
    f"select {TOKENS_VIEW}.*, {DOC_FREQ_VIEW}.df "
    f", row_number() OVER (PARTITION BY rid ORDER BY df, {TOKENS_VIEW}.token) as pos "
    f"from {TOKENS_VIEW}, {DOC_FREQ_VIEW} "
    f"where {TOKENS_VIEW}.token = {DOC_FREQ_VIEW}.token"
).execute(f'select * from {TOKENS_DOC_FREQ_VIEW}').fetchall()

[('db10_1', 4, 14, '53227250', 3, 1),
 ('db10_1', 4, 14, 'roche', 4, 2),
 ('db10_1', 4, 14, '3644', 12, 3),
 ('db10_1', 4, 14, 'rankin', 15, 4),
 ('db10_1', 4, 14, 'frenchs', 180, 5),
 ('db10_1', 4, 14, 'forest', 620, 6),
 ('db10_1', 4, 14, 'sophie', 1435, 7),
 ('db10_1', 4, 14, '9.0', 28728, 8),
 ('db10_1', 4, 14, '30.0', 74976, 9),
 ('db10_1', 4, 14, 'qld', 597882, 10),
 ('db10_1', 4, 14, '08', 856576, 11),
 ('db10_1', 4, 14, 'street', 2661790, 12),
 ('db10_1', 4, 14, '19340328.0', 25000001, 13),
 ('db10_1', 4, 14, '7577436', 25000001, 14),
 ('db10_1', 16, 12, 'mclachlan', 4, 1),
 ('db10_1', 16, 12, '4670', 48, 2),
 ('db10_1', 16, 12, '5.0', 22800, 3),
 ('db10_1', 16, 12, 'crescent', 409820, 4),
 ('db10_1', 16, 12, '07', 890190, 5),
 ('db10_1', 16, 12, '19772912.0', 25000001, 6),
 ('db10_1', 16, 12, '6495590', 25000001, 7),
 ('db10_1', 16, 12, '87605923', 25000001, 8),
 ('db10_1', 16, 12, 'auburh', 25000001, 9),
 ('db10_1', 16, 12, 'bissett', 25000001, 10),
 ('db10_1', 16, 12, 'rix',

In [109]:
_con.execute(
    f"select * from {DOC_FREQ_VIEW} "
    f"where df = {widow_placeholder}"
).fetchall()

[('aidon', 25000001),
 ('shsne', 25000001),
 ('edman', 25000001),
 ('19671108.0', 25000001),
 ('grcn', 25000001),
 ('08386325', 25000001),
 ('19881101.0', 25000001),
 ('84052052', 25000001),
 ('20260403', 25000001),
 ('09835132', 25000001),
 ('sylvznioa', 25000001),
 ('willeton', 25000001),
 ('89065392', 25000001),
 ('mccartthy', 25000001),
 ('smmth', 25000001),
 ('5633160', 25000001),
 ('terraces', 25000001),
 ('25685305', 25000001),
 ('7473388', 25000001),
 ('dickinson', 25000001),
 ('msw', 25000001),
 ('19220215.0', 25000001),
 ('2651', 25000001),
 ('3860', 25000001),
 ('yarragoh', 25000001),
 ('schiansky', 25000001),
 ('34222865', 25000001),
 ('maughan', 25000001),
 ('kralay', 25000001),
 ('rc', 25000001),
 ('mortiner', 25000001),
 ('19240904.0', 25000001),
 ('29010447', 25000001),
 ('claroke', 25000001),
 ('19450903.0', 25000001),
 ('3496', 25000001),
 ('shawm', 25000001),
 ('rondo', 25000001),
 ('09854162', 25000001),
 ('19220112.0', 25000001),
 ('72622577', 25000001),
 ('gevrgee

In [110]:
_con.execute(
    f"drop table if exists l_{PREFIXES_VIEW}"
).execute(
    f"create table l_{PREFIXES_VIEW} as "
    "select * "
    f"FROM {TOKENS_DOC_FREQ_VIEW} "
    f"where src = '{l_table}' "
    f"and rlen - pos + 1 >= (rlen * 2 * {t} / (1+{t})) "  # indexing prefix
).execute(
    f"drop table if exists r_{PREFIXES_VIEW}"
).execute(
    f"create table r_{PREFIXES_VIEW} as "
    "select * "
    f"FROM {TOKENS_DOC_FREQ_VIEW} "
    f"where src = '{r_table}' "
    f"and rlen - pos + 1 >= (rlen * 2 * {t} / (1+{t})) "  # indexing prefix
).execute(f"select * from l_{PREFIXES_VIEW}").fetchall()

[('db10_1', 4, 14, '53227250', 3, 1),
 ('db10_1', 4, 14, 'roche', 4, 2),
 ('db10_1', 4, 14, '3644', 12, 3),
 ('db10_1', 4, 14, 'rankin', 15, 4),
 ('db10_1', 4, 14, 'frenchs', 180, 5),
 ('db10_1', 4, 14, 'forest', 620, 6),
 ('db10_1', 4, 14, 'sophie', 1435, 7),
 ('db10_1', 4, 14, '9.0', 28728, 8),
 ('db10_1', 4, 14, '30.0', 74976, 9),
 ('db10_1', 4, 14, 'qld', 597882, 10),
 ('db10_1', 16, 12, 'mclachlan', 4, 1),
 ('db10_1', 16, 12, '4670', 48, 2),
 ('db10_1', 16, 12, '5.0', 22800, 3),
 ('db10_1', 16, 12, 'crescent', 409820, 4),
 ('db10_1', 16, 12, '07', 890190, 5),
 ('db10_1', 16, 12, '19772912.0', 25000001, 6),
 ('db10_1', 16, 12, '6495590', 25000001, 7),
 ('db10_1', 16, 12, '87605923', 25000001, 8),
 ('db10_1', 16, 12, 'auburh', 25000001, 9),
 ('db10_1', 27, 11, 'fisher', 6, 1),
 ('db10_1', 27, 11, 'secomb', 25, 2),
 ('db10_1', 27, 11, '3199', 54, 3),
 ('db10_1', 27, 11, 'thomas', 3286, 4),
 ('db10_1', 27, 11, '13.0', 13300, 5),
 ('db10_1', 27, 11, '23.0', 42427, 6),
 ('db10_1', 27, 1

In [111]:
l_widows = _con.execute(
    "select count(*) "
    f"from l_{PREFIXES_VIEW} "
    f"where src = '{l_table}' "
    f"and df = {widow_placeholder}"
).fetchall()[0][0]

r_widows = _con.execute(
    "select count(*) "
    f"from r_{PREFIXES_VIEW} "
    f"where src = '{r_table}' "
    f"and df = {widow_placeholder}"
).fetchall()[0][0]

r, s = (l_table, r_table) if l_widows > r_widows else (r_table, l_table)
r_pfx, s_pfx = (f'l_{PREFIXES_VIEW}', f'r_{PREFIXES_VIEW}') if l_widows > r_widows else (f'r_{PREFIXES_VIEW}', f'l_{PREFIXES_VIEW}')

_con.execute(
    f"drop table if exists {s_pfx}"
).execute(
    f"create table {s_pfx} as "
    "select * "
    f"FROM {TOKENS_DOC_FREQ_VIEW} "
    f"where src = '{s}' "
    f"and rlen - pos + 1 >= (rlen * {t}) "  # probing prefix
)

_con.execute(
    f"drop table if exists {PREFIXES_VIEW}"
).execute(
    f"create table {PREFIXES_VIEW} as "
    "select src, rid, rlen, token, pos "
    f"from {r_pfx} "
    "union "
    "select src, rid, rlen, token, pos "
    f"from {s_pfx}"
).execute(
    f"drop table if exists {r_pfx};"
    f"drop table if exists {s_pfx};"
).execute(
    f"select * from {PREFIXES_VIEW}"
).fetchall()

[('db10_1', 16, 12, '07', 5),
 ('db10_1', 30, 14, 'nsw', 8),
 ('db10_1', 45, 10, 'delprat', 2),
 ('db10_1', 45, 10, '27.0', 4),
 ('db10_1', 56, 13, 'isabella', 4),
 ('db10_1', 58, 13, 'mctaggart', 3),
 ('db10_1', 69, 12, '08', 8),
 ('db10_1', 74, 14, 'torrens', 2),
 ('db10_1', 111, 12, 'nsw', 8),
 ('db10_1', 111, 12, 'street', 9),
 ('db10_1', 117, 12, '07', 5),
 ('db10_1', 124, 13, 'nuriootpa', 1),
 ('db10_1', 124, 13, '80.0', 5),
 ('db10_1', 136, 16, 'warren', 4),
 ('db10_1', 148, 15, 'cooney', 1),
 ('db10_1', 148, 15, '3043', 4),
 ('db10_1', 163, 13, 'faunce', 2),
 ('db10_1', 177, 15, 'brighton', 3),
 ('db10_1', 177, 15, '25.0', 7),
 ('db10_1', 200, 12, 'chevalley', 1),
 ('db10_1', 200, 12, '6007586', 2),
 ('db10_1', 200, 12, 'lily', 8),
 ('db10_1', 214, 13, 'millie', 3),
 ('db10_1', 224, 12, 'court', 5),
 ('db10_1', 224, 12, '19210323.0', 9),
 ('db10_1', 233, 14, 'jayde', 1),
 ('db10_1', 233, 14, 'qld', 8),
 ('db10_1', 252, 10, '1.0', 5),
 ('db10_1', 253, 13, '02', 9),
 ('db10_1', 2

In [112]:
_con.execute(
    f"drop table if exists {CANDIDATE_SET_VIEW}"
).execute(
    f"CREATE table {CANDIDATE_SET_VIEW} AS ("
    "SELECT pr1.rid AS rid1, pr2.rid AS rid2 "
    ", MAX(pr1.pos) as maxPos1, MAX(pr2.pos) as maxPos2, count(*) as prOverlap "
    f"FROM {PREFIXES_VIEW} pr1, {PREFIXES_VIEW} pr2 "
    "WHERE pr1.token = pr2.token "
    f"and pr1.src = '{l_table}' "
    f"and pr2.src = '{r_table}'"
    # length filter
    f"AND pr1.rlen >= (pr2.rlen * {t})"
    f"AND pr2.rlen >= (pr1.rlen * {t})"
    # positional filter
    "AND LEAST((pr1.rlen - pr1.pos + 1), (pr2.rlen - pr2.pos + 1)) >= "
    f"((pr1.rlen + pr2.rlen) * {t} / (1 + {t})) "
    "GROUP BY pr1.rid, pr2.rid "
    ")"
).execute(f"select * from {CANDIDATE_SET_VIEW}").fetchall()

[(575, 9659, 8, 9, 1),
 (720, 9196, 6, 9, 1),
 (1165, 9659, 6, 9, 1),
 (1497, 7117, 8, 7, 2),
 (1648, 6952, 6, 9, 2),
 (1769, 7785, 9, 8, 1),
 (1887, 8405, 7, 8, 1),
 (3387, 6480, 6, 7, 1),
 (4118, 8742, 6, 6, 1),
 (3544, 8720, 9, 8, 2),
 (4515, 9196, 7, 9, 2),
 (898, 8720, 6, 7, 1),
 (1083, 7371, 10, 7, 1),
 (1716, 5155, 6, 7, 1),
 (4824, 7117, 7, 6, 1),
 (556, 9402, 5, 8, 1),
 (587, 5182, 4, 9, 1),
 (3029, 8742, 5, 6, 1),
 (4390, 5852, 9, 6, 1),
 (4019, 8450, 9, 7, 1),
 (3762, 9016, 9, 8, 2),
 (1921, 7992, 8, 9, 1),
 (45, 8956, 5, 6, 2),
 (111, 7947, 8, 9, 1),
 (2363, 5899, 10, 10, 2),
 (2793, 7947, 8, 9, 2),
 (4515, 8907, 7, 8, 1),
 (1083, 9470, 9, 8, 1),
 (2171, 7170, 10, 8, 1),
 (429, 7709, 10, 9, 2),
 (556, 8194, 7, 8, 2),
 (1494, 8459, 9, 8, 2),
 (2324, 5384, 9, 6, 1),
 (2369, 6092, 6, 9, 1),
 (2489, 8840, 7, 6, 1),
 (2907, 9858, 8, 4, 1),
 (3029, 7709, 6, 9, 2),
 (3134, 5384, 8, 6, 1),
 (2511, 8907, 8, 8, 1),
 (2742, 9858, 7, 4, 1),
 (253, 9517, 9, 6, 1),
 (1165, 8406, 6, 8, 1)

In [113]:
_con.execute(
    "SELECT pr1.rid AS rid1, pr2.rid AS rid2 "
    ", MAX(pr1.pos) as maxPos1, MAX(pr2.pos) as maxPos2, count(*) as prOverlap "
    f"FROM {PREFIXES_VIEW} pr1, {PREFIXES_VIEW} pr2 "
    "WHERE pr1.token = pr2.token "
    "GROUP BY pr1.rid, pr2.rid "
).fetchall()

[(58, 2989, 9, 9, 7),
 (177, 9507, 3, 4, 1),
 (200, 5180, 9, 10, 8),
 (200, 1058, 9, 7, 5),
 (214, 3814, 8, 9, 3),
 (575, 1826, 2, 1, 1),
 (628, 5558, 9, 10, 1),
 (692, 6713, 8, 6, 1),
 (726, 7476, 9, 6, 2),
 (754, 1169, 10, 6, 3),
 (796, 6693, 5, 4, 1),
 (867, 5893, 7, 6, 1),
 (990, 8292, 9, 9, 4),
 (1076, 5997, 8, 7, 2),
 (1125, 7069, 8, 7, 1),
 (1233, 9146, 10, 9, 1),
 (1337, 6737, 6, 6, 1),
 (1420, 9885, 6, 8, 1),
 (1454, 6895, 8, 10, 1),
 (1454, 5558, 9, 10, 1),
 (1497, 5578, 7, 11, 1),
 (1526, 8869, 9, 7, 1),
 (1572, 9773, 5, 8, 1),
 (1648, 7079, 4, 7, 1),
 (575, 5578, 8, 12, 2),
 (579, 5285, 1, 2, 1),
 (655, 655, 9, 9, 9),
 (1182, 8070, 9, 9, 6),
 (1211, 717, 7, 7, 2),
 (1224, 6885, 10, 7, 1),
 (1373, 5730, 8, 9, 3),
 (1655, 9986, 7, 10, 1),
 (1690, 3733, 2, 3, 1),
 (879, 6913, 6, 2, 1),
 (1151, 8881, 9, 8, 1),
 (1230, 9986, 7, 10, 2),
 (1026, 5558, 12, 10, 1),
 (1165, 7368, 4, 10, 1),
 (1075, 1075, 8, 8, 8),
 (1793, 3000, 10, 9, 3),
 (1793, 7254, 6, 5, 1),
 (1806, 9640, 8, 11, 

In [114]:
_con.execute(
    f"drop table if exists {out_table_name}"
).execute(
    f"create table {out_table_name} as "
    "select r1.rid as rid1, r2.rid as rid2 "
    f"from {TOKENS_DOC_FREQ_VIEW} r1, {TOKENS_DOC_FREQ_VIEW} r2, {CANDIDATE_SET_VIEW} c "
    "where c.rid1 = r1.rid "
    "and c.rid2 = r2.rid "
    "and r1.token = r2.token "
    "and r1.pos >= maxPos1 "
    "and r2.pos >= maxPos2 "
    "group by r1.rid, r2.rid, r1.rlen, r2.rlen, prOverlap "
    f"having count(*) + prOverlap - 1 >= ((r1.rlen + r2.rlen) * {t} / (1+{t}))"
).execute(
    f"select * from {out_table_name}"
).fetchall()

[(1684, 8982),
 (3433, 7387),
 (1018, 5423),
 (3331, 7273),
 (3047, 7626),
 (2341, 6494),
 (3801, 8520),
 (58, 8583),
 (815, 6966),
 (674, 6505),
 (3273, 8821),
 (3929, 7722),
 (3801, 9992),
 (1309, 9948),
 (1725, 7913),
 (3032, 8836),
 (835, 7610),
 (3989, 5169),
 (3905, 8866),
 (1506, 7698),
 (3123, 9794),
 (2635, 9386),
 (298, 9618),
 (1326, 9580),
 (2681, 8807),
 (3691, 8868),
 (792, 8284),
 (356, 7069),
 (4362, 9764),
 (2661, 8975),
 (3963, 9938),
 (160, 5370),
 (951, 9675),
 (4141, 5329),
 (4685, 9171),
 (528, 8324),
 (3967, 8369),
 (931, 6983),
 (1567, 8261),
 (337, 8815),
 (823, 8648),
 (4155, 8270),
 (1752, 5193),
 (4122, 8974),
 (1729, 6974),
 (2866, 9179),
 (2656, 8081),
 (3526, 5293),
 (3119, 7366),
 (4413, 6351),
 (626, 8939),
 (1560, 5508),
 (2196, 8801),
 (3207, 8448),
 (2101, 9206),
 (4997, 6878),
 (4358, 8866),
 (787, 8067),
 (3408, 5637),
 (518, 6560),
 (53, 6263),
 (88, 6101),
 (4145, 5712),
 (1377, 9874),
 (2879, 8081),
 (4040, 6425),
 (2277, 6316),
 (812, 7889),
 (

In [115]:
_con.execute(
    f"drop table if exists bf_{out_table_name}"
).execute(
    f"create table bf_{out_table_name} as "
    "select r1.rid as rid1, r2.rid as rid2 "
    ", count(*) as overlap "
    f", ((r1.rlen + r2.rlen) * {t} / (1+{t})) "
    f"from {TOKENS_VIEW} as r1, {TOKENS_VIEW} as r2 "
    "where r1.token = r2.token "
    f"and r1.src = '{l_table}' and r2.src = '{r_table}' "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    f"having count(*) >= ((r1.rlen + r2.rlen) * {t} / (1+{t}))"
).execute(f"select * from bf_{out_table_name}").fetchall()

[(1463, 5864, 5, 4.500000000000001),
 (1315, 9225, 5, 4.666666666666667),
 (2057, 7908, 4, 4.0),
 (2111, 6169, 4, 4.0),
 (3739, 8866, 5, 3.666666666666667),
 (3754, 6041, 10, 5.166666666666667),
 (3774, 7883, 9, 4.500000000000001),
 (3806, 7966, 7, 3.666666666666667),
 (3808, 5979, 4, 3.833333333333333),
 (3619, 6497, 5, 4.166666666666667),
 (3739, 5518, 4, 3.5000000000000004),
 (3742, 6238, 5, 3.833333333333333),
 (3851, 5701, 5, 4.500000000000001),
 (3619, 6939, 7, 4.333333333333334),
 (3676, 7351, 5, 4.166666666666667),
 (3767, 7917, 9, 4.166666666666667),
 (3693, 6220, 5, 4.833333333333333),
 (3806, 8821, 8, 4.166666666666667),
 (3764, 7774, 8, 4.666666666666667),
 (4456, 5969, 4, 4.0),
 (4514, 7107, 9, 4.500000000000001),
 (4736, 6195, 8, 4.333333333333334),
 (4545, 8845, 5, 5.0),
 (4545, 7409, 5, 5.0),
 (4428, 8652, 6, 3.833333333333333),
 (4579, 9593, 5, 4.666666666666667),
 (4438, 5239, 6, 4.333333333333334),
 (4521, 6771, 9, 5.5),
 (4492, 5986, 4, 3.5000000000000004),
 (4451, 

In [116]:
_con.execute(
    "select * "
    f"from {out_table_name} m "
    f"full outer join bf_{out_table_name} b "
    "on (b.rid1 = m.rid1 and b.rid2 = m.rid2) "
    "or (b.rid1 = m.rid2 and b.rid2 = m.rid1) "
    "where m.rid1 is null "
    "or b.rid1 is null"
).fetchall()

[]

In [117]:
_con.execute(
    f"drop table if exists {INPUT_TABLE};"
    f"drop table if exists {TOKENS_VIEW};"
    f"drop table if exists full_outer_{DOC_FREQ_VIEW};"
    f"drop table if exists {DOC_FREQ_VIEW};"
    f"drop table if exists {TOKENS_DOC_FREQ_VIEW};"
    f"drop table if exists {PREFIXES_VIEW};"
    f"drop table if exists {CANDIDATE_SET_VIEW};"
)

In [118]:
_con.execute("show tables").fetchall()

[('bf_matches',), ('db10_1',), ('db10_2',), ('db10gt',), ('matches',)]